# 🥇 Spectral Affinity: Ultimate GPU Harmonic Mixer (v3.0)

This is the most advanced version of the pipeline, featuring **GPU-Neural DSP**, **Persistent Caching**, and **Energy-Based Clustering**.

**🚀 Professional Features:**
- **💾 Persistent Intelligence:** Saves analysis to a JSON cache. Never re-analyze the same song twice.
- **🔥 GPU-Accelerated (nnAudio):** Real-time Neural CQT and Key detection on CUDA.
- **⚡ Ultra-Batching:** Processes the library in large batches for maximum GPU saturation.
- **📈 Energy Mapping:** Detects spectral energy to ensure your sets build up intensity correctly.
- **📦 Smart Sets:** Generates ~60min sets with harmonic flow and BPM/Energy progression.

---

### 1. 🛠️ Setup & High-Performance Engines

In [ ]:
import os
import warnings
import json
import torch
import torchaudio
import numpy as np
import librosa
import glob
import shutil
import re
import gc
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from IPython.display import HTML, FileLink, display

warnings.filterwarnings('ignore')

# 📦 Install Missing GPU libs
try:
    from nnAudio.Spectrogram import CQT1992v2
except:
    !pip install -q nnAudio
    from nnAudio.Spectrogram import CQT1992v2

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 ACCELERATOR: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")

### 2. 🧠 Neural Brain & Cache Logic

In [ ]:
class SpectralBrainGPU:
    def __init__(self, device='cuda', sr=22050, cache_file='spectral_cache.json'):
        self.device = device
        self.sr = sr
        self.cache_file = cache_file
        self.cache = self._load_cache()
        
        # Initialize Neural Layers
        self.cqt_layer = CQT1992v2(sr=self.sr, n_bins=84, bins_per_octave=12).to(self.device)
        
        # Key Profiles (Krumhansl-Schmuckler)
        major = torch.tensor([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], device=device)
        minor = torch.tensor([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17], device=device)
        
        self.profiles = torch.stack([torch.roll(major, i) for i in range(12)] + 
                                    [torch.roll(minor, i) for i in range(12)]).t()

    def _load_cache(self):
        if os.path.exists(self.cache_file):
            with open(self.cache_file, 'r') as f: return json.load(f)
        return {}

    def save_cache(self):
        with open(self.cache_file, 'w') as f: json.dump(self.cache, f)

    def process_batch(self, batch_data):
        # batch_data: list of (path, audio_numpy, duration)
        paths = [x[0] for x in batch_data]
        audios = [x[1] for x in batch_data]
        durations = [x[2] for x in batch_data]
        
        # GPU Matrix Prep
        max_len = 22050 * 120 # Cap analysis at 2 mins for speed/VRAM
        batch_tensor = torch.zeros(len(audios), max_len, device=self.device)
        for i, a in enumerate(audios):
            l = min(len(a), max_len)
            batch_tensor[i, :l] = torch.from_numpy(a[:l]).to(self.device)

        # 1. GPU CQT & Chroma
        with torch.no_grad():
            spec = self.cqt_layer(batch_tensor)
            # spec shape: (Batch, Bins, Time)
            # Extract Energy (RMS) for intensity mapping
            energy = spec.pow(2).mean(dim=(1, 2)).cpu().numpy()
            
            # Simple Chroma (sum octaves)
            chroma = spec.view(len(audios), 7, 12, -1).sum(dim=(1, 3))
            chroma = chroma / (chroma.norm(dim=1, keepdim=True) + 1e-6)
            
            # Key Match
            corrs = torch.matmul(chroma, self.profiles)
            best_idx = torch.argmax(corrs, dim=1).cpu().numpy()

        # 2. Results
        pc = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        batch_results = []
        
        for i in range(len(audios)):
            idx = best_idx[i]
            res = {
                'key': pc[idx % 12],
                'mode': 'major' if idx < 12 else 'minor',
                'energy': float(energy[i]),
                'duration': float(durations[i]),
                'path': paths[i]
            }
            # Fast BPM estimate on CPU while GPU is idle
            onset = librosa.onset.onset_strength(y=audios[i][:22050*60], sr=22050)
            res['bpm'] = float(librosa.beat.tempo(onset_envelope=onset, sr=22050)[0])
            batch_results.append(res)
            
        return batch_results

def get_camelot(key, mode):
    c_map = {
        ('B', 'major'): '01B', ('F#', 'major'): '02B', ('C#', 'major'): '03B', ('G#', 'major'): '04B',
        ('D#', 'major'): '05B', ('A#', 'major'): '06B', ('F', 'major'): '07B', ('C', 'major'): '08B',
        ('G', 'major'): '09B', ('D', 'major'): '10B', ('A', 'major'): '11B', ('E', 'major'): '12B',
        ('G#', 'minor'): '01A', ('D#', 'minor'): '02A', ('A#', 'minor'): '03A', ('F', 'minor'): '04A',
        ('C', 'minor'): '05A', ('G', 'minor'): '06A', ('D', 'minor'): '07A', ('A', 'minor'): '08A',
        ('E', 'minor'): '09A', ('B', 'minor'): '10A', ('F#', 'minor'): '11A', ('C#', 'minor'): '12A'
    }
    return c_map.get((key, mode.lower()), "00X")

def clean_name(n):
    n = os.path.basename(n)
    name = n.rsplit('.', 1)[0]
    name = re.sub(r"^[\w\-]+?-", "", name)
    name = re.sub(r"[\-\_\.]+?", " ", name).strip()
    return f"{name}.{n.rsplit('.', 1)[1]}"

### 3. 🚀 High-Speed Pipeline Execution

In [ ]:
# CONFIG
INPUT_DIR = "/kaggle/input/datasets/danieldobles/ost-songs"
OUTPUT_DIR = "/kaggle/working/pro_sets"
BATCH_SIZE = 24
TARGET_DUR = 60 * 60

brain = SpectralBrainGPU(device=device)

print("🔍 Scanning Library...")
paths = []
for ext in ['*.mp3', '*.wav', '*.flac', '*.m4a']:
    paths.extend(glob.glob(os.path.join(INPUT_DIR, "**", ext), recursive=True))
paths = list(set(paths))

# 1. Filter against Cache
to_analyze = [p for p in paths if p not in brain.cache]
print(f"📦 Total: {len(paths)} | 🧠 Cached: {len(paths)-len(to_analyze)} | 🚀 New: {len(to_analyze)}")

if to_analyze:
    chunks = [to_analyze[i:i+BATCH_SIZE] for i in range(0, len(to_analyze), BATCH_SIZE)]
    with ThreadPoolExecutor(max_workers=4) as pool:
        for chunk in tqdm(chunks, desc="🔥 Analying Batches"):
            # Load CPU
            def load(p):
                try: 
                    y, _ = librosa.load(p, sr=22050); return (p, y, len(y)/22050)
                except: return None
            
            batch_data = [x for x in list(pool.map(load, chunk)) if x is not None]
            if not batch_data: continue
            
            # GPU Crunch
            results = brain.process_batch(batch_data)
            for r in results:
                brain.cache[r['path']] = r
            brain.save_cache()
            gc.collect()

# 2. Final Logic (Clustering)
library = [brain.cache[p] for p in paths if p in brain.cache]
# Sorting: Camelot -> Energy -> BPM
library.sort(key=lambda x: (get_camelot(x['key'], x['mode']), x['energy'], x['bpm']))

if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR)

sets, current, dur = [], [], 0
for t in library:
    current.append(t)
    dur += t['duration']
    if dur >= TARGET_DUR:
        sets.append(current); current, dur = [], 0
if current: sets.append(current)

for i, s in enumerate(sets):
    s_dir = os.path.join(OUTPUT_DIR, f"SET_{str(i+1).zfill(2)}")
    os.makedirs(s_dir, exist_ok=True)
    for idx, t in enumerate(s):
        cam = get_camelot(t['key'], t['mode'])
        meta = f"[{cam} - {int(t['bpm'])}BPM]"
        shutil.copy2(t['path'], os.path.join(s_dir, f"{str(idx+1).zfill(2)} - {meta} {clean_name(t['path'])}"))

print(f"✅ SUCCESS! Generated {len(sets)} professional sets.")
zip_name = "SpectralAffinity_ProMixes.zip"
!zip -0 -rq {zip_name} pro_sets
display(HTML(f"<h3>🚀 <a href='{zip_name}' id='dl'>DOWNLOAD FINAL MIXES</a></h3>"))
display(HTML("<script>setTimeout(() => document.getElementById('dl').click(), 1000);</script>"))